In [1]:
from datasets import load_dataset

processed_dataset = load_dataset("ilhamxx/data_enroon")

In [ ]:
### import time
from sentence_transformers import util
from sentence_transformers import SentenceTransformer
from datasets import load_dataset
import pandas as pd

# Load the dataset
processed_dataset = load_dataset("ilhamxx/data_enroon")

#model1 = SentenceTransformer("all-MiniLM-L6-v2")
#model2 = SentenceTransformer("Snowflake/snowflake-arctic-embed-m-v1.5", model_kwargs=dict(add_pooling_layer=False),)

#model4 = SentenceTransformer("Snowflake/snowflake-arctic-embed-m-v1.5")
#model3 = SentenceTransformer("multi-qa-mpnet-base-dot-v1")
#model4 = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1")
#model5 = SentenceTransformer("liddlefish/privacy_embedding_rag_10k_base_15_final")
#model6 = SentenceTransformer("multi-qa-distilbert-cos-v1")
model3 = SentenceTransformer("avsolatorio/GIST-Embedding-v0")






In [ ]:
import torch
passages = processed_dataset['train']['message'] 

passage_embeddings = model3.encode(passages, convert_to_tensor=True) # dans search 
    
type(passage_embeddings)

# Save the embeddings

torch.save(passage_embeddings, 'GIST_embedd.pt')


### Collection for Gist_embedd


In [3]:
import torch
import pandas as pd

# Example dataset loading (adjust based on your actual data loading method)
dataset = processed_dataset["train"].to_pandas()  # Convert to DataFrame if using Hugging Face datasets

documents = []
metadatas = []
ids = []

# Iterate through the DataFrame to prepare documents, metadatas, and ids
for index, row in dataset.iterrows():
    message = row['message']
    documents.append(message)
    metadatas.append({'source': row['file']})
    ids.append(str(index + 1))
    

# Load the saved embeddings (assuming you saved them as a PyTorch tensor)
embeddings_tensor = torch.load("GIST_embedd.pt",map_location=torch.device('cpu'))



In [4]:
# Convert the tensor to a list of lists (if needed)
embeddings = embeddings_tensor.cpu().numpy().tolist()

print(f"First loaded embedding: {embeddings[0]}")

First loaded embedding: [-0.26020821928977966, -0.3577384054660797, -0.42784956097602844, 0.009518216364085674, 0.3624672293663025, -0.3641715347766876, -0.06981982290744781, 0.1425640732049942, 0.2710697054862976, 0.320745050907135, 0.06306835263967514, 0.06343412399291992, -0.1318431794643402, -0.039864249527454376, -0.022979529574513435, -0.391057550907135, 0.08569692820310593, -0.16384099423885345, -0.16153180599212646, -0.22662924230098724, -0.24559162557125092, 0.0964452251791954, 0.03446658328175545, 0.11380860954523087, -0.21642513573169708, 0.030222875997424126, -0.025476209819316864, 0.16913224756717682, -0.22955025732517242, -0.019527217373251915, -0.033088523894548416, -0.16021941602230072, 0.12286289036273956, 0.010220210067927837, -9.031147783389315e-05, -0.15134228765964508, 0.20857088267803192, 0.02946341224014759, -0.04252311959862709, 0.3231905996799469, -0.27903833985328674, -0.16888074576854706, -0.007692705374211073, 0.0033321944065392017, -0.4094528555870056, -0.0

## ADD the embeddings to chromaDB

In [2]:
import chromadb
import numpy as np

# Initialize ChromaDB client
client = chromadb.PersistentClient(path="data_embeddings")

# Create a collection for the Enron messages
collection_BERT = client.create_collection(
    name="ENRON_GIST",
    metadata={"hnsw:space": "cosine"} #dot product (ip) if the models use it
)

# Define the batch size
batch_size = 5000  # Adjust this size based on your needs and constraints

# Add embeddings to ChromaDB in batches
for i in range(0, len(documents), batch_size):
    end_index = min(i + batch_size, len(documents))
    batch_documents = documents[i:end_index]
    batch_embeddings = embeddings[i:end_index]
    batch_metadatas = metadatas[i:end_index]
    batch_ids = ids[i:end_index]

    collection_BERT.add(
        documents=batch_documents,
        embeddings=batch_embeddings,
        metadatas=batch_metadatas,
        ids=batch_ids
    )

print("Data GIST successfully added to ChromaDB.")



Data GIST successfully added to ChromaDB.
